## Импорты

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import LinearSVC, SVC
import pandas as pd
from sklearn.decomposition import NMF, TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler

import requests
import json
import pymorphy2
from time import sleep
import re
import bs4

import warnings
warnings.filterwarnings('ignore')

## Загрузка и очистка тестовых данных

In [2]:
test = []

with open('test.csv') as file:
    
    review_lines = []
    
    while True:

        line = file.readline()
        if not line:
            break
            
        if line.startswith('<review>'):
            review_lines.append(line[len('<review>'):].strip())
        elif '</review>' in line:
            test.append(" ".join(review_lines))
            review_lines = []
            file.readline()
            file.readline()
        else:
            review_lines.append(line.strip())

In [3]:
len(test)

100

## Парсинг отзывов на телефоны

Отзывы на мобильные телефоны возьмём с сайта hi-tech.mail.ru. Под положительными отзывами будем понимать отзывы с оценкой 4 или 5. Отрицательные отзывы имеют оценку 1, 2 или 3.

Для запуска вызвать функцию `parse_mail_ru`.

In [4]:
def parse_page(page):
    
    print("Parsing page %s" % page)
    
    page_X = []
    page_y = []
    
    req = requests.get(page)
    page_json = json.loads(req.text)
    if page_json and page_json['pager'] and page_json['pager']['next']:
        next_page = page_json['pager']['next']['url']
    else:
        next_page = None
    
    if not page_json or not page_json['data']:
        return (page_X, page_y, next_page)
    
    for review in page_json['data']:
        
        review_text = ""
        
        for text in review['text']:
            review_text += " " + text['text']
            
        # удаляем теги
        review_text = re.sub("<[^>]*>", " ", review_text).replace('\n', ' ').replace('\r', ' ').replace('&quot;', '"')
        
            
        stars = review['star']
        if stars == 0:
            continue
        
        review_rating = 0 if stars <= 3 else 1

        page_X.append(review_text)
        page_y.append(review_rating)

    return (page_X, page_y, next_page)

In [5]:
def parse_item(item, delay_seconds):
    
    base_url = "https://hi-tech.mail.ru"
    
    url_template = "/ajax/%s-catalog/feedback/list/"
    
    item_X = []
    item_y = []
    page = base_url + (url_template % item)
    
    while True:

        (page_X, page_y, next_page) = parse_page(page)
        sleep(delay_seconds)
        
        item_X += page_X
        item_y += page_y
        
        if not(next_page):
            break
            
        page = base_url + next_page

        
    return (item_X, item_y)
    

In [6]:
def parse_mail_ru():
    delay_seconds = 1

    for page_number in range(1, 20):
        train_X = []
        train_y = []

        url = "https://hi-tech.mail.ru/mobile-catalog/?page=%d" % page_number
        parser = bs4.BeautifulSoup(requests.get(url).text, 'lxml')
        for link in parser.findAll('a', attrs = {'class': 'p-catalog-card__link link-holder'}):
            item = int(re.search('.*_(\d*)-.*', link['href']).group(1))
            item_X, item_y = parse_item(item, delay_seconds)
            train_X += item_X
            train_y += item_y
            print("Total of %d reviews" % len(train_X))

        train = pd.DataFrame.from_dict({'text': train_X, 'label': train_y})
        train.to_csv("mailru_%d.csv" % page_number, sep = ',', index = False)
    
    frames = []
    for page_number in range(1, 20):
        frames.append(pd.read_csv("mailru_%d.csv" % page_number))
    train = pd.concat(frames)

Считываем готовые данные из файла:

In [7]:
train = pd.read_csv('mailru_all.csv', sep = ',')

Количество положительных отзывов:

In [8]:
len(train[train.label == 1])

11194

Количество отрицательных отзывов:

In [9]:
len(train[train.label == 0])

1836

Видим, что положительных отзывов существенно больше, значит, стоит предварительно балансировать выборку.

## Балансирование выборки

Сбалансируем выборку с помощью `RandomOverSampler` из `imblearn` и убедимся, что теперь количество элементов в классах одинаково:

In [10]:
def oversample(dataset):
    oversampler = RandomOverSampler()
    X = dataset.text.values.reshape(-1, 1)
    X_over, y_over = oversampler.fit_resample(X, dataset.label)
    X_over = X_over.reshape(1, X_over.shape[0])[0]
    
    X_lem = dataset.lemmatized.values.reshape(-1, 1)
    X_lem_over, y_over = oversampler.fit_resample(X_lem, dataset.label)
    X_lem_over = X_lem_over.reshape(1, X_lem_over.shape[0])[0]
    return (X_over, X_lem_over, y_over)

X_over, X_lem_over, y_over = oversample(train)

In [11]:
X_over.shape

(22388,)

In [12]:
len(list(filter(lambda y : y == 0, y_over)))

11194

In [13]:
len(list(filter(lambda y : y == 1, y_over)))

11194

## Выбор пайплайна

In [14]:
def train_cv(pipeline, name, X, y):
    accuracy = cross_val_score(pipeline, X, y, cv = 5).mean()
    print("%s: %f" % (name, accuracy))

In [15]:
for (pipeline, name) in [

    (Pipeline([
        ("vectorizer", CountVectorizer()), 
        ("classifier", LogisticRegression())
    ]), "CountVectorizer/LogisticRegression с параметрами по умолчанию"),

    (Pipeline([
        ("vectorizer", TfidfVectorizer()), 
        ("classifier", LogisticRegression())
    ]), "TfidfVectorizer/LogisticRegression с параметрами по умолчанию"),
    
    (Pipeline([
        ("vectorizer", TfidfVectorizer(ngram_range = (1, 2))), 
        ("classifier", LogisticRegression())
    ]), "TfidfVectorizer/LogisticRegression с униграммами и биграммами"),
    
    (Pipeline([
        ("vectorizer", CountVectorizer()), 
        ("transformer", TfidfTransformer()),
        ("classifier", LogisticRegression())
    ]), "CountVectorizer/TfidfTransformer/LogisticRegression с параметрами по умолчанию"),

    (Pipeline([
        ("vectorizer", CountVectorizer()), 
        ("classifier", LinearSVC())
    ]), "CountVectorizer/LinearSVC с параметрами по умолчанию"),

    (Pipeline([
        ("vectorizer", CountVectorizer()), 
        ("classifier", SGDClassifier())
    ]), "CountVectorizer/SGDClassifier с параметрами по умолчанию"),

    (Pipeline([
        ("vectorizer", CountVectorizer(min_df = 50)), 
        ("classifier", SGDClassifier())
    ]), "CountVectorizer/SGDClassifier с min_df = 50"),

    (Pipeline([
        ("vectorizer", CountVectorizer()), 
        ("transformer", TfidfTransformer()),
        ("classifier", LinearSVC())
    ]), "CountVectorizer/TfidfTransformer/LinearSVC с параметрами по умолчанию"),

    (Pipeline([
        ("vectorizer", CountVectorizer()), 
        ("transformer", TfidfTransformer()),
        ("classifier", SGDClassifier())
    ]), "CountVectorizer/TfidfTransformer/SGDClassifier с параметрами по умолчанию"),
    
    (Pipeline([
        ("vectorizer", CountVectorizer(min_df = 10)), 
        ("classifier", LogisticRegression())
    ]), "CountVectorizer/LogisticRegression с min_df = 10"),
    
    (Pipeline([
        ("vectorizer", CountVectorizer(min_df = 50)), 
        ("classifier", LogisticRegression())
    ]), "CountVectorizer/LogisticRegression с min_df = 50"),
    
        (Pipeline([
        ("vectorizer", CountVectorizer(max_df = 0.1)), 
        ("classifier", LogisticRegression())
    ]), "CountVectorizer/LogisticRegression с max_df = 0.1"),

    (Pipeline([
        ("vectorizer", CountVectorizer(max_df = 0.5)), 
        ("classifier", LogisticRegression())
    ]), "CountVectorizer/LogisticRegression с max_df = 0.5"),
    
    (Pipeline([
        ("vectorizer", CountVectorizer(max_df = 0.8)), 
        ("classifier", LogisticRegression())
    ]), "CountVectorizer/LogisticRegression с max_df = 0.8"),
    
    (Pipeline([
        ("vectorizer", CountVectorizer(ngram_range = (2, 2))), 
        ("classifier", LogisticRegression())
    ]), "CountVectorizer/LogisticRegression с биграммами"),

    (Pipeline([
        ("vectorizer", CountVectorizer(ngram_range = (1, 2))), 
        ("classifier", LogisticRegression())
    ]), "CountVectorizer/LogisticRegression с униграммами и биграммами"),

    (Pipeline([
        ("vectorizer", CountVectorizer(max_df = 0.5, ngram_range = (1, 2))), 
        ("classifier", LogisticRegression())
    ]), "CountVectorizer/LogisticRegression с max_df = 0.5, униграммами и биграммами"),

    (Pipeline([
        ("vectorizer", CountVectorizer(max_df = 0.1, ngram_range = (1, 2))), 
        ("classifier", LogisticRegression())
    ]), "CountVectorizer/LogisticRegression с max_df = 0.1, униграммами и биграммами"),
    


]:
    train_cv(pipeline, name, X_over, y_over)

CountVectorizer/LogisticRegression с параметрами по умолчанию: 0.977310
TfidfVectorizer/LogisticRegression с параметрами по умолчанию: 0.963954
TfidfVectorizer/LogisticRegression с униграммами и биграммами: 0.976416
CountVectorizer/TfidfTransformer/LogisticRegression с параметрами по умолчанию: 0.963954
CountVectorizer/LinearSVC с параметрами по умолчанию: 0.977310
CountVectorizer/SGDClassifier с параметрами по умолчанию: 0.982536
CountVectorizer/SGDClassifier с min_df = 50: 0.960783
CountVectorizer/TfidfTransformer/LinearSVC с параметрами по умолчанию: 0.982446
CountVectorizer/TfidfTransformer/SGDClassifier с параметрами по умолчанию: 0.978292
CountVectorizer/LogisticRegression с min_df = 10: 0.976684
CountVectorizer/LogisticRegression с min_df = 50: 0.955914
CountVectorizer/LogisticRegression с max_df = 0.1: 0.977712
CountVectorizer/LogisticRegression с max_df = 0.5: 0.977354
CountVectorizer/LogisticRegression с max_df = 0.8: 0.977310
CountVectorizer/LogisticRegression с биграммами: 

Выберем `TfidfVectorizer` с униграммами и биграммами и классификатор `LinearSVC`. Ограничим количество признаков.

In [20]:
pipeline = Pipeline([
         ("vectorizer", TfidfVectorizer(ngram_range = (1, 2), max_features = 10000)),
         ("classifier", LinearSVC(random_state = 42))
     ])

pipeline.fit(X_over, y_over)

test_df = pd.DataFrame(test)
test_df.columns = ['text']
test_df['Id'] = range(0, 100)
test_df['y_int'] = pipeline.predict(test_df.text)
test_df['y'] = test_df.apply (lambda row: 'pos' if row.y_int == 1 else 'neg', axis = 1)
test_df.drop(['text', 'y_int'], axis = 1).to_csv('results.csv', sep = ',', index = False)

Посмотрим на результаты.

In [21]:
test_df

text  Id  y_int    y
0   Ужасно слабый аккумулятор, это основной минус ...   0      0  neg
1   постоянно долбиться в уши и звонить по паре ча...   1      1  pos
2   К сожалению, факт поломки через месяц использо...   2      0  neg
3   Есть телефоны намного лучше. НЕ БЕРИТЕ !!! Я п...   3      1  pos
4   Удобная Клавиатура и русская раскладка - 2 сим...   4      1  pos
..                                                ...  ..    ...  ...
95  У меня уже он 2 года. Покупался за 8500, сейча...  95      0  neg
96  Вчера купил мне очень нравится!Замечательный т...  96      1  pos
97  Телефон, если им пользоваться, вынуждает на ру...  97      1  pos
98  Очень должно ждала этот телефон,когда появится...  98      1  pos
99  Приятный внешний вид и все. Убогость ОС убивае...  99      1  pos

[100 rows x 4 columns]

Результаты на Kaggle:
![kaggle.png](kaggle.png)